In [1]:
import pandas as pd

df= pd.read_csv('article_highlights.csv')

In [2]:
df.head()

,url,article,highlights
0,https://www.dailymail.co.uk/tvshowbiz/article-...,Beyoncé showcases her incredible figure in plu...,Beyoncé has shown off her flawless beauty in a...
1,https://www.dailymail.co.uk/tvshowbiz/article-...,Radio 1 listeners in shock as sex noises are p...,BBC Radio 1 listeners were left choking on the...
2,https://www.dailymail.co.uk/tvshowbiz/article-...,"TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, ...",Dan Edgar and Ella Rae Wise put on a loved-up ...
3,https://www.dailymail.co.uk/tvshowbiz/article-...,Bradley Cooper recalls 'crazy' pitch meeting a...,Bradley Cooper discussed the 'crazy' experienc...
4,https://www.dailymail.co.uk/tvshowbiz/article-...,Margaret Qualley and Beanie Feldstein stun in ...,Margaret Qualley and Beanie Feldstein were dre...


In [3]:
df.tail()

,url,article,highlights
8171,https://www.dailymail.co.uk/sport/football/art...,Bernardo Silva's calamitous free kick against ...,Bernardo Silva's calamitous free-kick against ...
8172,https://www.dailymail.co.uk/sport/football/art...,Son Heung-min and Lee Kang-in bury the hatchet...,Spurs star Son Heung-min and PSG's Lee Kang-in...
8173,https://www.dailymail.co.uk/sport/football/art...,Bernardo Silva's calamitous free kick against ...,Bernardo Silva's calamitous free-kick against ...
8174,https://www.dailymail.co.uk/sport/football/art...,Son Heung-min and Lee Kang-in bury the hatchet...,Spurs star Son Heung-min and PSG's Lee Kang-in...
8175,https://www.dailymail.co.uk/sport/football/art...,Bernardo Silva's calamitous free kick against ...,Bernardo Silva's calamitous free-kick against ...


In [4]:
df.size

24528

In [5]:
df.shape

(8176, 3)

In [6]:
df.drop('url', axis=1, inplace= True)
df.head()

,article,highlights
0,Beyoncé showcases her incredible figure in plu...,Beyoncé has shown off her flawless beauty in a...
1,Radio 1 listeners in shock as sex noises are p...,BBC Radio 1 listeners were left choking on the...
2,"TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, ...",Dan Edgar and Ella Rae Wise put on a loved-up ...
3,Bradley Cooper recalls 'crazy' pitch meeting a...,Bradley Cooper discussed the 'crazy' experienc...
4,Margaret Qualley and Beanie Feldstein stun in ...,Margaret Qualley and Beanie Feldstein were dre...


In [8]:
df.shape

(8176, 2)

In [9]:
#basic cleaning of the columns
import re
from bs4 import BeautifulSoup 

def clean_text(text):
    if not isinstance(text, str):  
        return ""
    
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub(r'\[[0-9]*\]', '', text)
    text = re.sub(r"[^a-zA-Z]", " ", text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    
    return text.strip()

df['article'] = df['article'].apply(clean_text)
df['highlights'] = df['highlights'].apply(clean_text)

df.head()

,article,highlights
0,beyonc showcases her incredible figure in plun...,beyonc has shown off her flawless beauty in a ...
1,radio listeners in shock as sex noises are pla...,bbc radio listeners were left choking on their...
2,towie s dan edgar and ella rae wise put on a l...,dan edgar and ella rae wise put on a loved up ...
3,bradley cooper recalls crazy pitch meeting at ...,bradley cooper discussed the crazy experience ...
4,margaret qualley and beanie feldstein stun in ...,margaret qualley and beanie feldstein were dre...


In [10]:
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

nlp = spacy.load("en_core_web_sm")

tqdm.pandas()

# Define the extractive summarization function
def extractive_summary(text, n=1):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    
    if len(sentences) <= n:
        return ' '.join(sentences)
    
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    sentence_scores = np.array(X.sum(axis=1)).flatten()
    top_n_idx = sentence_scores.argsort()[-n:][::-1]
    
    return ' '.join([sentences[i] for i in sorted(top_n_idx)])

# Define a safe summary function to handle long texts
def safe_summary(text, max_chars=5000):
    if len(text) > max_chars:
        text = text[:max_chars]
    return extractive_summary(text)

df['extractive_summary'] = df['article'].progress_apply(safe_summary)
print("Original:\n", df.iloc[0]['article'][:1000])
print("\nExtractive Summary:\n", df.iloc[0]['extractive_summary'])
doc = nlp(df.iloc[0]['article'])
print(f"Number of sentences: {len(list(doc.sents))}")

C:\Users\ihase\anaconda3\envs\summarizer\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
100%|██████████████████████████████████████████████████████████████████████████████| 8176/8176 [03:26<00:00, 39.50it/s]

Original:
 beyonc showcases her incredible figure in plunging white dress with thigh high leg slit as she promotes her new haircare brand c credbeyonc has shown off her flawless beauty in a new photo to promote her brand new haircare range

Extractive Summary:
 beyonc showcases her incredible figure in plunging white dress with thigh high leg slit as she promotes her new haircare brand c credbeyonc has shown off her flawless beauty in a new photo to promote her brand new haircare range
Number of sentences: 1


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def abstractive_summary(text, max_input_length=1024):
    if len(text) > max_input_length:
        text = text[:max_input_length]
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
    return summary[0]['summary_text']

df['abstractive_summary'] = df['article'].progress_apply(abstractive_summary)


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

C:\Users\ihase\anaconda3\envs\summarizer\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ihase\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
